In [ ]:
import tweepy
from TweetTransformations import checkClaims, translate_to_german,  translate_to_english, deconstructTwitterQueryResponse
from config import access_token, access_token_secret, bearer_token, consumer_key, consumer_secret


In [ ]:
api = tweepy.Client(
    access_token=access_token,
    access_token_secret=access_token_secret,
    bearer_token=bearer_token,
    consumer_key=consumer_key,
    consumer_secret=consumer_secret,
    wait_on_rate_limit=False)

In [ ]:
# keyword search for klimawandel 
response = api.search_recent_tweets(query="Klimawandel -is:retweet", expansions=['referenced_tweets.id'])

In [ ]:
response.data[0].id

In [ ]:
tweet_list = deconstructTwitterQueryResponse(response)

In [ ]:
tweet_list

In [ ]:
translated_tweet_list = []
for tweet in tweet_list:
    tweet_id = tweet.get("tweet_id")
    original_tweet= tweet.get("original_tweet")
    translated_original_tweet = await translate_to_english(original_tweet)
    answer_dict = {
                "original_tweet" : original_tweet,
                "tweet_id": tweet_id,
                "translated_original_tweet" : translated_original_tweet,
            }
    translated_tweet_list.append(answer_dict)

In [ ]:
translated_tweet_list

In [ ]:
list_of_factchecked_tweets = await checkClaims(tweetlist=translated_tweet_list)

In [ ]:
for tweet in list_of_factchecked_tweets:
    original_tweet = tweet.get("original_tweet")
    answer_tweet = tweet.get("answer_tweet")
    tweet_id = tweet.get("tweet_id")
    translated_answer_tweet = await translate_to_german(answer_tweet)
    print(tweet_id)
    # post tweet
    try:
        api.create_tweet(text=translated_answer_tweet,in_reply_to_tweet_id=tweet_id)
    except tweepy.errors.Forbidden as e:
        print(f"error while posting: {e}")
